### Coding with LLMs


In [101]:
# Ooodles of Imports
import os
import json
import re
from icecream import ic
from rich.console import Console
from rich import print
from typing import List
from pydantic import BaseModel
from loguru import logger
import pudb
from typing_extensions import Annotated

console = Console()
from langchain.llms import GPT4All
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from typing import Any, Optional
from langchain.output_parsers.openai_functions import OutputFunctionsParser
from langchain.schema import FunctionMessage


from langchain.schema import (
    Generation,
    OutputParserException,
)
from IPython.display import Image
from IPython.core.display import HTML 

In [ ]:
# Useful helpers
def model_to_openai_function(cls):
    return {"name": cls.__name__, "parameters": cls.model_json_schema()}


class JsonOutputFunctionsParser2(OutputFunctionsParser):
    """Parse an output as the Json object."""

    def parse_result(self, result: List[Generation]) -> Any:
        function_call_info = super().parse_result(result)
        if self.args_only:
            try:
                # Waiting for this to merge upstream
                return json.loads(function_call_info, strict=False)
            except (json.JSONDecodeError, TypeError) as exc:
                raise OutputParserException(
                    f"Could not parse function call data: {exc}"
                )
        function_call_info["arguments"] = json.loads(function_call_info["arguments"])
        return function_call_info


from IPython.display import display, HTML


def print_line():
    display(HTML("<hr>"))


def print_prompt(prompt):
    print("Prompt:")
    for m in prompt.messages:
        print(f"{type(m)}  {m.prompt}")
def print_function_call(response):
    # #     additional_kwargs={'function_call': {'name': 'GetWeather', 'arguments': '{\n"City": "Spain"\n}'}},
    function = response.additional_kwargs["function_call"]
    print_line()
    print (f"Call: {function['name']}")
    print (function['arguments'])
    

### Langchain - Super cool, we'll use it, but not our focus

### Prompts And Models - CPU and compilation

In [ ]:
# Everyone wants to be a comedian
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessage, 
    AIMessage,
    HumanMessage
)

joke_prompt = ChatPromptTemplate.from_template("tell me {count} jokes about {topic}")
print(joke_prompt.messages)

In [ ]:
# Compile the program and run on a familiar GPU

model = ChatOpenAI()
chain = joke_prompt | model

# Run it => Invoke()
topic = "Software Engineers"
count = 2
result = chain.invoke({"topic": topic, "count": count})

# Show output
print(result)
print_line()
print(result.content)


In [102]:
# Compile the program and run on our CPU

local_model = GPT4All(model="./llama2.bin")
local_chain = joke_prompt | local_model

# Run it => Invoke()
topic = "Software Engineers"
count = 2
result = local_chain.invoke({"topic": topic, "count": count})

# Show output
print_line()
print(result)

Found model file at  ./llama2.bin
falcon_model_load: loading model from './llama2.bin' - please wait ...
falcon_model_load: n_vocab   = 65024
falcon_model_load: n_embd    = 4544
falcon_model_load: n_head    = 71
falcon_model_load: n_head_kv = 1
falcon_model_load: n_layer   = 32
falcon_model_load: ftype     = 2
falcon_model_load: qntvr     = 0
falcon_model_load: ggml ctx size = 3872.64 MB
falcon_model_load: memory_size =    32.00 MB, n_mem = 65536
falcon_model_load: ........................ done
falcon_model_load: model size =  3872.59 MB / num tensors = 196


Sure, here are two jokes about software engineers:

1. Why did the software engineer quit her job?

She couldn't handle the stress of deadlines and code reviews!

2. How do you make a software engineer feel appreciated?

Write them a thank-you note for all the hard work they put into making your life easier.

### Functions - The I/O System

In [ ]:
# The rain in spain
# Tell the model the "OS" supports getting the weather


# define a callable function
class GetWeather(BaseModel):
    City: str


get_weather = model_to_openai_function(GetWeather)

weather_prompt_template = "What's the weather in {place}"
model = ChatOpenAI()
weather_prompt = ChatPromptTemplate.from_template(weather_prompt_template)

chain = weather_prompt | model.bind(
    functions=[get_weather]  # tell model we can call it.
)


print ("Show the full response:")
response = chain.invoke({"place": "Spain"})
print(response)

print ("Include an output parser in the chain:")
print_function_call(response)


In [ ]:
# Woah  - Did you see  the bug?
model = ChatOpenAI(model="gpt-4")

# Do "some more programming"
weather_prompt = ChatPromptTemplate(
    messages=[
        SystemMessage(
            content="When an API takes a city, infer an appropritiate city"
        ),
        HumanMessagePromptTemplate.from_template(weather_prompt_template),
    ]
)
chain = weather_prompt | model.bind(functions=[get_weather])
response = chain.invoke({"place": "Spain"})
print("Output")
print(response)
print_line()
print (response.additional_kwargs)

In [ ]:
# Back to our functions

weather_with_data = weather_prompt.copy()

# Update prompt with AI's desire to call a function
weather_with_data.append(response)

# Need to make tomorrow's cut, just stamp this please :)
# Will come back and make a dispatcher and call actual functions

weather_with_data.append(
    FunctionMessage(name="GetWeather", content="5 degrees and rainy")
)

print(weather_with_data)


chain = weather_with_data | model.bind(
    functions=[get_weather]
)  # tell model we can call it.

response = chain.invoke({"place": "Spain"})
print_line()
print(response)

In [ ]:
# Innovate - Why do we seperate view from model?


class Joke(BaseModel):
    setup: str
    punch_line: str
    reason_joke_is_funny: str


class GetJokes(BaseModel):
    count: int
    jokes: List[Joke]


get_jokes = model_to_openai_function(GetJokes)

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me {count} jokes about {topic}")
chain = prompt | model.bind(functions=[get_jokes]) | JsonOutputFunctionsParser2()
print(prompt.messages)
response = chain.invoke({"topic": topic, "count": count})
print("Output")
print(response)

In [ ]:
# Innovate - What's better then doing math with a calculator?


solve_math_with_python = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "Write code to solve the users problem. the last line of the python  program should print the answer. Do not use sympy"
        ),
        HumanMessagePromptTemplate.from_template("What is the 217th prime"),
    ]
)


class ExecutePythonCode(BaseModel):
    valid_python: str
    code_explanation: str


model = ChatOpenAI(model="gpt-4").bind(
    function_call={"name": "ExecutePythonCode"},  # tell gpt to use this model
    functions=[model_to_openai_function(ExecutePythonCode)],
)


# JsonOutputFunctionParser2 == PrettyPrintOutput

chain = solve_math_with_python | model | JsonOutputFunctionsParser2()
response = chain.invoke({})

print(response["code_explanation"])
print_line()

valid_python = response["valid_python"]
print(valid_python)

In [ ]:
# Woah Nelly  ... are you sure you want to do this??
print_line()
# input("Are you sure you want to run this code??")
exec(valid_python)

### Memory - Staying in RAM

In [ ]:
# Basics of Conversational Memory - # no memory
model = ChatOpenAI().bind(temperature=0)
prompt_template = "Tell me another joke"
last_prompt = ""
for i in range(4):
    print_line()
    prompt = ChatPromptTemplate.from_template(prompt_template)
    print(f"Prompt {i}\n", prompt.messages)
    chain = prompt | model
    response = chain.invoke({})

    print(f"Response {i}\n", response.content)

In [ ]:
# The memory module!

from langchain.memory import ChatMessageHistory

memory = ChatMessageHistory()

# memory.save_context({"input":"I like software engineer jokes"} , {"output":"That's odd, but fine"})
memory.add_user_message("I like software engineering jokes")
memory.add_ai_message("OK!")
print(memory.messages)

In [ ]:
# Basics of Conversational Memory w/Memory

memory = ChatMessageHistory()
memory.add_user_message("I like software engineering jokes")
memory.add_ai_message("OK!")

# human always says the same thing
human_says = "tell me another joke"
prompt = ""  # have it outside loop so we can print it
for i in range(4):
    print_line()

    # build prompt, we'll print the last one ...
    prompt = ChatPromptTemplate.from_messages(memory.messages)
    prompt.append(human_says)
    # run the chain
    chain = prompt | model

    response = chain.invoke({})
    print(f"Response {i}\n", response.content)

    # store history
    memory.add_user_message(human_says)
    memory.add_ai_message(response.content)

print_line()
print_line()
print(f"Final Prompt\n", prompt.messages)

In [ ]:
# Innovate - A new kind of lossy compression

semantic_compression_prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a compressing chat model, summarize the entire conversation into a paragaph"
        )
    ]
)

for m in memory.messages:
    semantic_compression_prompt.append(m)

chain = semantic_compression_prompt | model

response = chain.invoke({})
print("Semantic Compression of Conversation \n", response.content)

### Retrievel and Index - Exceeding RAM and Disk

In [ ]:
# Raw Data -> Index -> Retreive:  A picture is worth a thousand words
Image(filename="./images/retrieval_pipeline.jpg")

In [126]:
# Find some young pets - using english

# Data Set small enough we can use raw data run the query and retrieval in one fell swoop
things_we_saw = [
    "dog", "cat","zebra", "puppy","calf", "puppies" , "kitten",  "cow", "desk","rubber band","mouse", "airpods", "smelly socks", "sandals","jacket", "cats", "young dog", "young cat", "young hamster"
]

# The query to retreive  - but not super
retrieval_query="puppy|kitten"
young_pets = [ item for item in things_we_saw if re.match(retrieval_query, item) ]


print (young_pets)

['puppy', 'kitten']

In [127]:
# Find some young pets - using english

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma, FAISS

# Reperesent words using embedding (if time see pictures)
VectorStore =  FAISS
embeddings = OpenAIEmbeddings()
db = VectorStore.from_texts(things_we_saw, embedding=embeddings)

In [ ]:
print(db.similarity_search_with_relevance_scores("young dogs", k=5))

In [ ]:
# Retrieve using LLM 

In [ ]:
# Great Walk through of Word Embeddings

https://investigate.ai/text-analysis/word-embeddings/


### Planning - Workflow and Verifcation and Agents

In [ ]:
# Help me write a talk

our_task = "Write a talk on coding with large language models in markdown"

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template(our_task)
result = (prompt | model) .invoke({})
print(result.content)

In [118]:
# Chain of thought, 
model = ChatOpenAI()
prompt = ChatPromptTemplate(
                                messages=[
        SystemMessagePromptTemplate.from_template(
            "Given a task, make a list of steps to accomplish it"
        ),
        HumanMessagePromptTemplate.from_template(our_task)])
result = (prompt | model) .invoke({})
the_plan = result.content
print(the_plan)

1. Start by introducing the topic of coding with large language models and its significance in the field of 
artificial intelligence.
2. Explain what large language models are and how they have revolutionized natural language processing tasks.
3. Discuss the benefits of using large language models in coding, such as automated code generation, code 
completion, and bug detection.
4. Explain the different approaches to coding with large language models, including fine-tuning, prompt 
engineering, and zero-shot learning.
5. Provide examples of popular large language models like GPT-3, GPT-2, and T5, and discuss their applications in 
coding.
6. Discuss the challenges and limitations of using large language models in coding, such as bias, ethics, and the 
need for careful evaluation.
7. Present case studies or real-world examples where coding with large language models has been successful.
8. Discuss potential future developments and advancements in coding with large language models, such as model 
optimization and hybrid approaches.
9. Provide practical tips and best practices for developers who want to start coding with large language models, 
including data preprocessing, model selection, and fine-tuning techniques.
10. Discuss the potential impact of coding with large language models on the software development industry and the 
job market.
11. Conclude the talk by summarizing the main points discussed and emphasizing the importance of staying updated 
with the latest advancements in coding with large language models.
12. Open the floor for questions and engage in a discussion with the audience to address any concerns or queries 
they may have.

In [124]:
model = ChatOpenAI()

prompt = ChatPromptTemplate(
                                    messages=[
        SystemMessage( content=
            "Given a task an d a plan make recommendation changes with reasoning. First message will be the task, the next the plan"
        ),
        HumanMessage(content=our_task),
        HumanMessage(content=the_plan)])
result = (prompt | model) .invoke({})
the_plan = result.content
print(the_plan)

Your plan for the talk on coding with large language models in markdown is comprehensive and covers all the 
necessary points. However, I would like to suggest a few changes to enhance the clarity and effectiveness of the 
talk:

1. Start by introducing the topic of coding with large language models and its significance in the field of 
artificial intelligence and software development.

2. Explain what large language models are and how they have revolutionized natural language processing tasks, 
particularly in the context of coding.

3. Discuss the benefits of using large language models in coding, such as automated code generation, code 
completion, and bug detection, and provide concrete examples to illustrate these benefits.

4. Explain the different approaches to coding with large language models, including fine-tuning, prompt 
engineering, and zero-shot learning, and highlight their respective strengths and limitations.

5. Provide examples of popular large language models like GPT-3, GPT-2, and T5, and discuss their specific 
applications in coding, such as generating code snippets, assisting with documentation, and facilitating code 
reviews.

6. Discuss the challenges and limitations of using large language models in coding, such as bias, ethics, and the 
need for careful evaluation, and emphasize the importance of responsible and ethical use of these models.

7. Present case studies or real-world examples where coding with large language models has been successful, 
showcasing the practical benefits and impact on software development.

8. Discuss potential future developments and advancements in coding with large language models, such as model 
optimization techniques, hybrid approaches combining traditional programming with language models, and 
collaborative coding with multiple models.

9. Provide practical tips and best practices for developers who want to start coding with large language models, 
including data preprocessing, model selection, and fine-tuning techniques, and recommend available resources and 
tools for further exploration.

10. Discuss the potential impact of coding with large language models on the software development industry and the 
job market, addressing both the opportunities and potential challenges that may arise.

11. Conclude the talk by summarizing the main points discussed and emphasizing the importance of continuous 
learning and adaptation to the evolving landscape of coding with large language models.

12. Open the floor for questions and engage in a discussion with the audience to address any concerns or queries 
they may have, encouraging them to share their experiences and perspectives on the topic.

By incorporating these changes, your talk will be more engaging, informative, and impactful for the audience. Good 
luck with your presentation!